In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from looptools.components import MokuPIDSymbolicController, MokuPIDController

In [ ]:
sps = 80e6
frfr = np.logspace(np.log10(1e-1), np.log10(1e6), int(1e5))
frfr = frfr[0:-1]

moku_pid = MokuPIDController("Moku PID", sps, -10, 1e3, None, 1e5, f_trans=1e2)
moku_pid_sym = MokuPIDSymbolicController("Moku PID Symbolic", sps, -10, 1e3, None, 1e5)

ax = moku_pid.bode_plot(frfr, label='Numeric')
moku_pid_sym.bode_plot(frfr, axes=ax, label='Symbolic', ls='--')
plt.show()

In [ ]:
def moku_pid_tf_model(f):
    """
    Transfer function model taken from:
    https://liquidinstruments.com/case-studies/precision-laser-noise-suppression-in-optical-phase-locking/
    """
    f = np.asarray(f)
    omega = 2 * np.pi * f  # Convert frequency to angular frequency [rad/s]
    s = 1j * omega

    # Transfer function C(s) = (9.4e6 / s^2) * (1 + s/1e4) * (1 + s/500)
    H = (9.4e6 / s**2) * (1 + s / 1e4) * (1 + s / 500)

    mag_db = 20 * np.log10(np.abs(H))
    phase_deg = np.angle(H, deg=True)

    return mag_db, phase_deg


# Get target data
target_mag_db, target_phase_deg = moku_pid_tf_model(frfr)

# Objective function
def objective(params):
    Kp_dB, Fc_i, Fc_ii = params
    pid = MokuPIDController("Moku PID", sps, Kp_dB, Fc_i, Fc_ii, None, f_trans=1e2)
    H = pid.TF(frfr)
    
    model_mag_db = 20 * np.log10(np.abs(H))
    model_phase_deg = np.angle(H, deg=True)

    mag_error = model_mag_db - target_mag_db
    phase_error = model_phase_deg - target_phase_deg

    # Weighting (optional tuning knobs)
    weight_mag = 1.0
    weight_phase = 1.0

    error = weight_mag * mag_error**2 + weight_phase * phase_error**2
    return np.sqrt(np.mean(error))

# Initial guess and bounds: (Kp_dB, Fc_i, Fc_ii)
x0 = [0.0, 100.0, 1.0]  # [dB, Hz, Hz]
bounds = [(-40, 40), (1.0, 1e5), (1e-1, 1e4)]

# Optimization
result = minimize(objective, x0, bounds=bounds, method='L-BFGS-B')
Kp_dB_fit, Fc_i_fit, Fc_ii_fit = result.x

print(f"Fitted parameters:")
print(f"  Kp_dB  = {Kp_dB_fit:.4f} dB")
print(f"  Fc_i   = {Fc_i_fit:.4f} Hz")
print(f"  Fc_ii  = {Fc_ii_fit:.4f} Hz")

# Evaluate fitted model
pid_fit = MokuPIDController("Fitted PID", sps, Kp_dB_fit, Fc_i_fit, Fc_ii_fit, None, f_trans=1e2)

fig, ax = plt.subplots(2,1, figsize=(5,5), dpi=150)
pid_fit.bode_plot(frfr, axes=ax, label='Fit', dB=True)
ax[0].semilogx(frfr, target_mag_db, label='Model', ls='--')
ax[1].semilogx(frfr, target_phase_deg, ls='--')
ax[0].legend()
plt.show()